# Practica del mundo real.

Eres un empleado de __Empresa Energética SA__, y estais preparando los datos de todos los sistemas para pasarlos a un modelo de predicción. Te han encargado una tarea, cargar los datos de los clientes de Pueblo Paleta, un pequeño pueblo en el que hay __740 casas__.

Los datos que tienes que cargar son los datos del contador de cada vivienda, que genera un registro de datos cada hora. En este registro se encuentra el consumo acumulado de la última hora de cada vivienda.  
El problema es que el concentrador energético de pueblo paleta es muy antiguo y sus sistema informático aún más. Su base de datos es una versión muy prehistórica de las bases de datos actuales y tiene muchas limitaciones, como por ejemplo la forma en la que almacenan los datos. La base de datos es un conjunto de filas y columnas no numeradas, en la que puede haber máximo 16 registros por fila, pero puede haber filas infinitas. En una misma fila no se puede guardar 24 registros, solo guardan 16. Los ingenieros de aquella epoca hicieron un apaño para guardar esa información, y es que cada día se irían guardando los datos de cada vecino de forma consecutiva y ordenada. 

Primero el vecino1, luego el vecino2, luego el vecino3 y así hasta el vecino740. Además tampoco podemos identificar el a que día corresponde cada conjunto de registros, pero si sabemos que el primer día de datos corresponde al 16 de enero de 2004.
Nos han asegurado que no hay ningun registro nulo en todo el histórico de datos del sistema, puesto que aunque es viejísimo es muy eficaz.

Tu tarea será:
- Organizar esta información de forma lógica, es decir, cada fila corresponde a un día del año, y cada columna corresponde a un cliente. Y en cada celda se encontrará una lista de los registros que marca el contador. 
- Crear un nuevo dataset de información (otra matriz), en la que en vez de tener los registros, tengamos la media de estos registros por cliente y día. 
- Sabiendo que la energía cuesta 0.25€ el KW/h, crear una nueva matriz en la que por cada cliente y día, tengamos el coste total de cada cliente, para tener la facturación diaria de estos.

Mucho ánimo.

In [2]:
import numpy as np
import pandas as pd

In [3]:
# Generar los datos

np.random.seed(2023)
data = np.random.randint(low=12,high=150,size=(999000,16))
data

array([[ 99,  37,  15, ...,  75,  41,  61],
       [126, 124,  61, ...,  46,  61,  89],
       [127, 127,  99, ..., 145,  19,  95],
       ...,
       [ 45,  21,  35, ..., 118, 125,  41],
       [ 74,  93,  67, ...,  48,  23,  27],
       [ 68,  73,  45, ..., 137,  40,  32]])

In [44]:
# Reorganizar los datos de manera lógica:
#   Primera dimensión (900) se corresponde con el total de días del conjunto de datos
#   Segunda dimensión (740) se corresponde con el número de vecinos
#   Tercera dimensión (24) se corresponde con los registros diarios, uno cada hora (24 al día)

reshaped_data = data.reshape(-1,740,24)
reshaped_data

array([[[ 99,  37,  15, ..., 123, 139, 100],
        [ 84,  77,  34, ..., 145,  19,  95],
        [ 67, 137,  18, ...,  83,  83,  48],
        ...,
        [ 88,  56,  40, ..., 120, 140,  38],
        [144,  80,  79, ..., 133,  94,  40],
        [ 65,  56,  92, ...,  13,  32,  15]],

       [[ 97, 103, 113, ...,  65,  67,  75],
        [140,  56,  91, ...,  82, 121,  77],
        [125,  49,  59, ...,  53,  68,  24],
        ...,
        [ 98, 143,  81, ...,  48,  23, 125],
        [ 23,  23,  55, ..., 113,  61, 122],
        [ 27,  19,  30, ...,  40,  43,  39]],

       [[ 44,  61,  45, ...,  79,  28,  36],
        [ 45,  32,  69, ...,  46,  39, 128],
        [ 56,  78, 144, ..., 112, 126,  58],
        ...,
        [114,  81,  29, ..., 144, 102,  77],
        [110, 144,  66, ...,  89, 148,  22],
        [133, 107,  83, ..., 138,  92,  48]],

       ...,

       [[ 78, 130,  33, ...,  97,  38, 140],
        [130,  70,  99, ..., 114, 114,  91],
        [120,  68, 103, ...,  65, 107,  38

In [45]:
# Consumo medio por cliente y día

def consumo_medio(datos):
    '''
    Función que calcula el consumo medio por cliente y día
        Args: 
            datos (numpy array 3D): datos con el consumo energético por horas
        Returs:
            consumo_medio(numpy array): media del consumo energético diario por cliente
    '''
    media_consumo = np.zeros(datos.shape[:-1])

    for dia in range(len(datos)):
        for cliente in range(len(datos[dia])):
            media_consumo[dia][cliente] = np.mean(datos[dia][cliente])
    return media_consumo 

In [47]:
# Array con las medias diarias de cada cliente de su consumo energético. 
# Registros de 900 días y 740 clientes
datos_medios = consumo_medio(reshaped_data)
datos_medios 

array([[81.125     , 85.08333333, 80.375     , ..., 91.875     ,
        80.33333333, 75.875     ],
       [76.625     , 77.45833333, 77.75      , ..., 89.54166667,
        71.25      , 60.41666667],
       [74.375     , 82.95833333, 77.125     , ..., 88.83333333,
        87.04166667, 84.04166667],
       ...,
       [84.875     , 93.08333333, 76.79166667, ..., 76.125     ,
        83.29166667, 70.16666667],
       [84.54166667, 85.75      , 69.70833333, ..., 69.79166667,
        73.375     , 65.375     ],
       [81.79166667, 94.58333333, 76.58333333, ..., 88.20833333,
        72.5       , 66.75      ]])

In [9]:
# Coste del consumo energético por día

def consumo_diario(datos,tarifa):
    '''
    Función que calcula el consumo total en euros (€) por cliente y día
        Args: 
            datos(numpy array): datos con el consumo energético por horas
            tarifa (float): precio en euros (€) del kW/h
        Returs (numpy array): 
            coste (numpy array): coste en euros (€) del consumo energético por cliente y día
    '''
    costes = np.zeros(datos.shape[:-1])

    for dia in range(len(datos)):
        for cliente in range(len(datos[dia])):
            consumo_dia = np.sum(datos[dia][cliente]) # Suma de los registros de consumo diario (24 registros)
            coste = consumo_dia*tarifa
            costes[dia][cliente] = coste
    return costes

In [42]:
# Array con las el coste del consumo energético diario de cada cliente.
# Registros de 900 días y 740 clientes

facturacion_diaria = consumo_diario(reshaped_data,0.25)

In [43]:
facturacion_diaria

array([[486.75, 510.5 , 482.25, ..., 551.25, 482.  , 455.25],
       [459.75, 464.75, 466.5 , ..., 537.25, 427.5 , 362.5 ],
       [446.25, 497.75, 462.75, ..., 533.  , 522.25, 504.25],
       ...,
       [509.25, 558.5 , 460.75, ..., 456.75, 499.75, 421.  ],
       [507.25, 514.5 , 418.25, ..., 418.75, 440.25, 392.25],
       [490.75, 567.5 , 459.5 , ..., 529.25, 435.  , 400.5 ]])